ПАКЕТ SURPRISE

    используйте данные MovieLens 1M
    можно использовать любые модели из пакета
    получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М можешь не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете


Коллаборативная фильтрация: нужно предсказать оценку фильма, использую коллаборцию по фильмам или по юзерам

In [2]:
! pip install surprise

In [5]:
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise import accuracy
from tqdm import tqdm_notebook
import pandas as pd

In [6]:
ratings = pd.read_csv('/home/benito/Desktop/Введение и классификация рекомендательных систем/Архив/ratings.csv')

Нужна матрица с оценками по фильмам и пользователям

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
del ratings['timestamp']

In [9]:
ratings.columns=['uid','iid','rating']

In [10]:
ratings['rating'].max()

5.0

In [11]:
ratings['rating'].min()

0.5

In [12]:
reader=Reader(rating_scale=(0.5,5))

In [13]:
dataset=Dataset.load_from_df(ratings,reader)

In [14]:
kfold=KFold(5)

In [15]:
def get_scores(algo):
    scores = []
    for trainset, testset in tqdm_notebook(kfold.split(dataset)):
        algo.fit(trainset)
        predictions=algo.test(testset)
        scores.append(accuracy.mse(predictions))
    return scores

In [16]:
algo_user=KNNBasic(k=40,sim_options={'name':'cosine','user_based': True})
scores_algo_user = get_scores(algo_user)

Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.9495
Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.9377
Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.9600
Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.9529
Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.9400



In [17]:
algo_item=KNNBasic(k=40,sim_options={'name':'pearson_baseline','user_based': False})
scores_algo_item = get_scores(algo_item)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.8437
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.8373
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.8361
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.8483
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.8492



In [19]:
algo_user_means=KNNWithMeans(k=40,sim_options={'name':'cosine','user_based': True})
scores_algo_user_means = get_scores(algo_user_means)

Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.8015
Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.7998
Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.8240
Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.8188
Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 0.8160



In [20]:
algo_item_means=KNNWithMeans(k=40,sim_options={'name':'pearson_baseline','user_based': False})
scores_algo_item_means = get_scores(algo_item_means)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.7808
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.7628
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.7828
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.7800
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
MSE: 0.7871



In [29]:
sum(scores_algo_item_means)/len(scores_algo_item_means)

0.7787081430307123

In [31]:
print ('Mean RMSE for user:',sum(scores_algo_user)/len(scores_algo_user))
print ('Mean RMSE for item:',sum(scores_algo_item)/len(scores_algo_item))
print ('Mean RMSE for user_with_means:',sum(scores_algo_user_means)/len(scores_algo_user_means))
print ('Mean RMSE for item_with_means:',sum(scores_algo_item_means)/len(scores_algo_item_means))

Mean RMSE for user: 0.9480464402650306
Mean RMSE for item: 0.8429131387181231
Mean RMSE for user_with_means: 0.8120291243988553
Mean RMSE for item_with_means: 0.7787081430307123
